In [1]:
# 1. Creating a line/plane that divides the observations with widest possible margin

In [2]:
# 2. The vector between the decision boundery and the observations that influence the decision boundary

In [3]:
# 3. Since SVM tries to find the largest boudary which is influenced by the observations actual values (it will use features with bigger scale since they can produce bigger boundary)

In [4]:
# 4. It can use the distance from boundary as a confidence score, it cannot output probability directly

In [5]:
# 5. Use primal form, dual form is slower

In [6]:
# 6. Increase gamma and C to try a better fit

In [7]:
# 7. I should not

In [8]:
# 8.

In [1]:
# Import iris dataset
from sklearn import datasets

iris = datasets.load_iris()
x = iris['data'][:, (2,3)]
y = iris['target']

setosa_or_versicolor = (y==0) | (y==1)
x = x[setosa_or_versicolor]
y = y[setosa_or_versicolor]

In [2]:
# Libraries
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import SGDClassifier
import numpy as np
from scipy.stats import reciprocal, uniform
np.random.seed(42)

In [3]:
# Common hypperparameters
C = 5
alpha = 1/(C*len(x))
max_iter=1000
tol = 0.001

In [4]:
# LinearSVC
lsvc_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_svc', LinearSVC(C=C, loss='hinge', max_iter=max_iter, tol=tol)),
])
lsvc_clf.fit(x, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linear_svc',
                 LinearSVC(C=5, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.001,
                           verbose=0))],
         verbose=False)

In [5]:
# SGDClassifier
svm_clf_sgd = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_sgdc', SGDClassifier(loss='hinge', alpha=alpha, max_iter=max_iter, tol=tol))
])
svm_clf_sgd.fit(x, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linear_sgdc',
                 SGDClassifier(alpha=0.002, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=1000, n_iter_no_change=5, n_jobs=None,
                               penalty='l2', power_t=0.5, random_state=None,
                               shuffle=True, tol=0.001, validation_fraction=0.1,
                               verbose=0, warm_start=False))],
         verbose=False)

In [6]:
# SVM
svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_svm', SVC(kernel='linear', C=C, max_iter=max_iter, tol=tol))
])
svm_clf.fit(x, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linear_svm',
                 SVC(C=5, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='linear', max_iter=1000,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [7]:
# Coefs
print('LinearSVC:', lsvc_clf['linear_svc'].intercept_, lsvc_clf['linear_svc'].coef_)
print('SGDClassifier:', svm_clf_sgd['linear_sgdc'].intercept_, svm_clf_sgd['linear_sgdc'].coef_)
print('SVM:', svm_clf['linear_svm'].intercept_, svm_clf['linear_svm'].coef_)

LinearSVC: [0.2848045] [[1.05353738 1.09896137]]
SGDClassifier: [0.62357843] [[1.39532881 1.67285447]]
SVM: [0.31896852] [[1.1203284  1.02625193]]


In [8]:
# 9.
from sklearn.model_selection import RandomizedSearchCV

In [9]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True)

x = mnist['data']
y = mnist['target'].astype(np.uint8)

x_train = x[:60000]
y_train = y[:60000]
x_test = x[10000:]
y_test = y[10000:]

In [10]:
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(x_train, y_train)

D:\anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0)

In [ ]:
svm_clf = SVC()

param_grid = {
    'kernel': ['rbf', 'poly'],
    'gamma': reciprocal(0.001, 0.1),
    'C': uniform(1, 10),
}

svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rand_search', RandomizedSearchCV(svm_clf, param_grid, n_iter=10, cv=3,
                                      scoring='neg_mean_squared_error',
                                      return_train_score=True,
                                      verbose=2, refit=True))
])

svm_pipeline.fit(x_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=6.142344384136116, gamma=0.01530485212183146, kernel=rbf ......


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [23]:
#svm_clf = SVC()

#param_grid = {
#    'svm_clf__kernel': ['rbf', 'poly'],
#    'svm_clf__gamma': reciprocal(0.001, 0.1),
#    'svm_clf__C': uniform(2, 10),
#}

#svm_pipeline = Pipeline([
#    ('scaler', StandardScaler()),
#    ('svm_clf', SVC())
#])

#svm_rand_search = RandomizedSearchCV(svm_pipeline, param_grid, n_iter=10, cv=3,
#                                   scoring = 'neg_mean_squared_error',
#                                   return_train_score=True,
#                                   verbose=2, refit=True)



In [ ]:
svm_pipeline['rand_search'].best_estimator_